In [ ]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [ ]:
all_data = []

for i in tqdm(range(1,52), desc="Fetching Articles"):
    url = f"https://push.api.bbci.co.uk/batch?t=%2Fdata%2Fbbc-morph-lx-commentary-data-paged%2Fabout%2F8467c0e0-584b-41de-9682-756b311216b5%2FisUk%2Ffalse%2Flimit%2F20%2FnitroKey%2Flx-nitro%2FpageNumber%2F{i}%2Fversion%2F1.5.6?timeout=5"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        results= data['payload']
        for result in results:
            all_data.extend(result['body']['results'])
df = pd.DataFrame(all_data)
df.dropna(subset='url',inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.shape

In [ ]:
articles = {
    'Title': [],
    'Description': []
}
URL='https://www.bbc.com/'
for index, row in tqdm(df.iterrows(), total=len(df)):
    link = URL + row['url']
    response = requests.get(link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        title = soup.find('h1').get_text()
        paragraphs = soup.find_all("p")
        description = '\n'.join([paragraph.get_text() for paragraph in paragraphs])
        articles['Title'].append(title)
        articles['Description'].append(description)

In [ ]:
news_df = pd.DataFrame(articles)
news_df['Label'] = 1
news_df.to_csv('bbc_news.csv',index=False)